In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import re

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import fasttext
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
import warnings
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np
import neattext.functions as nfx
 
warnings.filterwarnings(action='ignore')

from code_utils.utils import preprocess,aplatir

In [2]:
df_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_ipcc.jsonl', lines= True).dropna(subset=['year'])

In [3]:
df_not_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_not_ipcc.jsonl', lines= True)

In [4]:
df_ipcc=df_ipcc[['year','doi', 'title','topics','locations_names','locations_ids']]

In [5]:
df_not_ipcc=df_not_ipcc[['year','doi', 'title','topics','locations_names','locations_ids']]

In [6]:
df_ipcc=df_ipcc.dropna(subset=['year','doi', 'title','topics','locations_names','locations_ids']).drop_duplicates(subset=['doi'])

In [7]:
df_not_ipcc=df_not_ipcc.dropna(subset=['doi', 'title','topics','locations_names','locations_ids'])

In [8]:
data_counts=pd.Series([int(x) for x in list(df_ipcc.year.dropna())]).value_counts()
data_counts=pd.Series(dict(data_counts)).sort_index()

building the learning dataset

In [9]:
print(len(df_ipcc),len(df_not_ipcc))

48219 48219


In [10]:
df_ipcc['label']='ipcc'
df_not_ipcc['label']='not_ipcc'

In [11]:
df_all=pd.concat([df_ipcc,df_not_ipcc]).reset_index()
del df_all['index']

In [12]:
df_all.loc[:,'doi']=df_all.loc[:,'doi'].apply(lambda x: str(nfx.remove_stopwords(x)).replace('https://doi.org/','').lower())

In [13]:
df_all.loc[:,'title']=df_all.loc[:,'title'].apply(lambda x: str(nfx.remove_stopwords(x)).replace('}','').replace('{','').lower())

In [14]:
df_all.loc[:,'topics']=df_all.loc[:,'topics'].apply(lambda x: nfx.remove_stopwords(' '.join(x)).lower())

In [15]:
df_all

,year,doi,title,topics,locations_names,locations_ids,label
0,2019.0,10.3389/978-2-88963-118-6,oceanobs'19: ocean opportunity. volume 1,marine biodiversity ecosystem functioning,Frontiers research topics,1664-8714,ipcc
1,1900.0,10.1002/andp.19003081208,"\""uber die bedeutung des wasserdampfes und der...",stratospheric chemistry climate change impacts...,Annalen der Physik,0003-3804,ipcc
2,1964.0,10.3402/tellusa.v16i1.8885,the parameters atmospheric turbidity,dynamics ocean surface waves wind interaction,Tellus A,0280-6495,ipcc
3,1929.0,10.1080/20014422.1929.11880498,on atmospheric transmission sun radiation dust...,aerosols' impact climate hydrological cycle,Geografiska Annaler,1651-3215,ipcc
4,2013.0,10.1002/rog.20022,a review global ocean temperature observations...,oceanic modeling circulation studies global se...,Reviews of Geophysics,1944-9208,ipcc
...,...,...,...,...,...,...,...
96433,1967.0,10.1007/bf02349230,played physicians patient's path mental hospital,crisis resolution home treatment teams mental ...,Community mental health journal,0010-3853,not_ipcc
96434,1966.0,10.1038/210701a0,delayed fluorescence chlorophyll vitro vivo,molecular mechanisms photosynthesis photoprote...,Nature,0028-0836,not_ipcc
96435,1966.0,10.1507/endocrine1927.42.8_880,子宮筋homogenateにおけるacetylcholineと組織蛋白との結合に及ぼすoxy...,principles stereochemical structure nomenclatu...,Nippon Naibunpi Gakkai zasshi/Nihon Naibunpi G...,0029-0661,not_ipcc
96436,1686.0,10.1098/rstl.1686.0008,"voyage emperour china eastern tartary, anno 1682",linguistic studies turkic languages silk roads...,Philosophical transactions of the Royal Societ...,0261-0523,not_ipcc


In [16]:
df_all.columns

Index(['year', 'doi', 'title', 'topics', 'locations_names', 'locations_ids',
       'label'],
      dtype='object')

using fasttext

In [ ]:
df_all['label'] = '__label__' + df_all['label'].astype(str)
df_all['category_description']=df_all['label']+' '+df_all['title']+' '+df_all['topics']+' '+df_all['locations_names']+' '+df_all['locations_ids']

In [18]:
train, test = train_test_split(df_all, test_size=0.2, random_state=42)

In [19]:
train.shape, test.shape

((77150, 8), (19288, 8))

In [20]:
train.to_csv("teds.train", columns=["category_description"], index=False, header=False)
test.to_csv("teds.test", columns=["category_description"], index=False, header=False)

In [21]:
model = fasttext.train_supervised(input="teds.train")
model.test("teds.test")

(16040, 0.9733790523690773, 0.9733790523690773)

In [22]:
model.save_model("fasttext_model_teds_no_topics1202.bin")

In [23]:
model = fasttext.load_model("fasttext_model_teds_no_topics1202.bin")

In [24]:
test_data = pd.read_csv("teds.test", header=None, names=["text"])

In [25]:
true_labels = []
predicted_labels = []

for line in test_data["text"]:
    label, text = line.split(' ', 1)
    true_labels.append(label)
    text = text.replace('\n', '').strip()
    predicted_label = model.predict(text)[0][0]  
    predicted_labels.append(predicted_label)

In [26]:
conf_matrix = confusion_matrix(true_labels, predicted_labels)

In [27]:
conf_matrix

array([[9346,  348],
       [ 250, 9344]], dtype=int64)

In [28]:
accuracy_score(true_labels, predicted_labels)

0.9689962671090834

In [29]:
recall_score(true_labels, predicted_labels, average="weighted")

0.9689962671090834

In [30]:
f1_score(true_labels, predicted_labels, average="weighted")

0.9689962834432007

In [46]:
t=test.iloc[3,:]

In [47]:
t

year                                                               2018.0
doi                                              10.3389/fgene.2018.00558
title                   persistence epigenomic effects recovery repeat...
topics                  real-time polymerase chain reaction mycotoxins...
locations_names                                     Frontiers in genetics
locations_ids                                                   1664-8021
label                                                   __label__not_ipcc
category_description    __label__not_ipcc persistence epigenomic effec...
Name: 50590, dtype: object

In [48]:
print(f"{t.title} {t.topics} {t.locations_names} {t.locations_ids}",model.predict(nfx.remove_stopwords(f"{t.title} {t.topics} {t.locations_names} {t.locations_ids}"), k=-1))

persistence epigenomic effects recovery repeated treatment nephrocarcinogens real-time polymerase chain reaction mycotoxins impact genotoxicity carcinogenesis mechanisms Frontiers in genetics 1664-8021 (('__label__not_ipcc', '__label__ipcc'), array([0.9973914 , 0.00262858]))
